<a href="https://colab.research.google.com/github/siva4iov/Amazon_Employee_Access_Challenge/blob/main/Amazon_com_Employee_Access_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
print(train.size)

327690


In [ ]:
print(test.size)

589210


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train.drop(labels=["ACTION"], axis=1), train["ACTION"], test_size=0.33,
                                                  shuffle=True, stratify=train["ACTION"], random_state=42)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedShuffleSplit

In [ ]:
stratss = StratifiedShuffleSplit(test_size=.2, train_size=.5, n_splits=3)
param_grid = {"var_smoothing":10.**np.arange(-9, 2)}
gs = GridSearchCV(GaussianNB(), param_grid, cv=5)
gs.fit(X_train, y_train)
print("Best params: ",gs.best_params_)
print("Best score: ", gs.best_score_)

Best params:  {'var_smoothing': 0.1}
Best score:  0.9421088590298338


In [ ]:
X0=X_val[y_val==0]
y0=y_val[y_val==0]

In [ ]:
print("Score on eval set:")
gs.score(X_val,y_val)

Score on eval set:


0.9421120769373035

In [ ]:
print("Score on eval set for y==0:")
gs.score(X0, y0)

Score on eval set for y==0:


0.0

In [ ]:
pd.concat((y_train, y_val)).value_counts(normalize=True)

1    0.94211
0    0.05789
Name: ACTION, dtype: float64

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold

In [ ]:
skf = StratifiedKFold(shuffle=True,n_splits=3)
param_grid = {"learning_rate":10.**np.arange(-2, 1), "n_estimators":[100, 500, 1000]}
gs = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=skf, verbose=3)
gs.fit(X_train, y_train)
print("Best params: ",gs.best_params_)
print("Best score: ", gs.best_score_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3] END learning_rate=0.01, n_estimators=100;, score=0.942 total time=   3.7s
[CV 2/3] END learning_rate=0.01, n_estimators=100;, score=0.942 total time=   3.5s
[CV 3/3] END learning_rate=0.01, n_estimators=100;, score=0.942 total time=   4.4s
[CV 1/3] END learning_rate=0.01, n_estimators=500;, score=0.942 total time=  22.8s
[CV 2/3] END learning_rate=0.01, n_estimators=500;, score=0.943 total time=  21.9s
[CV 3/3] END learning_rate=0.01, n_estimators=500;, score=0.942 total time=  23.9s
[CV 1/3] END learning_rate=0.01, n_estimators=1000;, score=0.942 total time=  45.1s
[CV 2/3] END learning_rate=0.01, n_estimators=1000;, score=0.943 total time=  43.6s
[CV 3/3] END learning_rate=0.01, n_estimators=1000;, score=0.942 total time=  43.9s
[CV 1/3] END learning_rate=0.1, n_estimators=100;, score=0.942 total time=   4.4s
[CV 2/3] END learning_rate=0.1, n_estimators=100;, score=0.943 total time=   4.3s
[CV 3/3] END learning_rate

In [ ]:
skf = StratifiedKFold(shuffle=True,n_splits=5)
param_grid = {"learning_rate":10.**np.arange(-2, 1), "n_estimators":[100, 500, 1000]}
gs = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=skf, verbose=3,
#                   scoring=["precision", "recall", "f1"], refit="f1")
                  scoring="roc_auc")
gs.fit(X_train, y_train)
print("Best params: ",gs.best_params_)
print("Best score: ", gs.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END learning_rate=0.01, n_estimators=100;, score=0.684 total time=   4.1s
[CV 2/5] END learning_rate=0.01, n_estimators=100;, score=0.648 total time=   3.2s
[CV 3/5] END learning_rate=0.01, n_estimators=100;, score=0.663 total time=   3.3s
[CV 4/5] END learning_rate=0.01, n_estimators=100;, score=0.633 total time=   3.9s
[CV 5/5] END learning_rate=0.01, n_estimators=100;, score=0.717 total time=   4.8s
[CV 1/5] END learning_rate=0.01, n_estimators=500;, score=0.750 total time=  22.6s
[CV 2/5] END learning_rate=0.01, n_estimators=500;, score=0.708 total time=  21.0s
[CV 3/5] END learning_rate=0.01, n_estimators=500;, score=0.707 total time=  17.2s
[CV 4/5] END learning_rate=0.01, n_estimators=500;, score=0.701 total time=  19.3s
[CV 5/5] END learning_rate=0.01, n_estimators=500;, score=0.756 total time=  19.8s
[CV 1/5] END learning_rate=0.01, n_estimators=1000;, score=0.769 total time=  36.2s
[CV 2/5] END learning_rate

In [ ]:
from sklearn.metrics import accuracy_score
print("Score on eval set for y==0:")

accuracy_score(y0, gs.predict(X0))

Score on eval set for y==0:


0.16932907348242812

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
overs = SMOTE(k_neighbors=4)
X_train_res, y_train_res = overs.fit_resample(X_train, y_train)

In [ ]:
skf = StratifiedKFold(shuffle=True,n_splits=5)
param_grid = {"learning_rate":10.**np.arange(-2, 0), "n_estimators":[100, 500, 1000]}
gs = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=skf, verbose=3,
                  scoring="roc_auc")
gs.fit(X_train_res, y_train_res)
print("Best params: ",gs.best_params_)
print("Best score: ", gs.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END learning_rate=0.01, n_estimators=100;, score=0.757 total time=   5.9s
[CV 2/5] END learning_rate=0.01, n_estimators=100;, score=0.779 total time=   5.6s
[CV 3/5] END learning_rate=0.01, n_estimators=100;, score=0.762 total time=   5.6s
[CV 4/5] END learning_rate=0.01, n_estimators=100;, score=0.772 total time=   5.8s
[CV 5/5] END learning_rate=0.01, n_estimators=100;, score=0.758 total time=   5.6s
[CV 1/5] END learning_rate=0.01, n_estimators=500;, score=0.850 total time=  29.4s
[CV 2/5] END learning_rate=0.01, n_estimators=500;, score=0.868 total time=  29.3s
[CV 3/5] END learning_rate=0.01, n_estimators=500;, score=0.853 total time=  29.5s
[CV 4/5] END learning_rate=0.01, n_estimators=500;, score=0.861 total time=  31.0s
[CV 5/5] END learning_rate=0.01, n_estimators=500;, score=0.856 total time=  30.9s
[CV 1/5] END learning_rate=0.01, n_estimators=1000;, score=0.902 total time=  59.7s
[CV 2/5] END learning_rate

In [ ]:
print("Score on eval set for y==0:")

accuracy_score(y0, gs.predict(X0))

Score on eval set for y==0:


0.365814696485623

In [ ]:
print("Score on eval set:")
gs.score(X_val, y_val)

Score on eval set:


0.7910659787684816

In [ ]:
pred = gs.predict_proba(test.drop(["id"], axis=1))[:,1]
res = pd.concat([test["id"], pd.Series(pred, name="Action")], axis=1)
print(res)
res.to_csv("submissions.csv", index=False)

          id    Action
0          1  0.537106
1          2  0.873251
2          3  0.918410
3          4  0.901321
4          5  0.975759
...      ...       ...
58916  58917  0.927484
58917  58918  0.902622
58918  58919  0.628685
58919  58920  0.897471
58920  58921  0.981242

[58921 rows x 2 columns]


## Kaggle score :
# 0.80229

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
model = CatBoostClassifier(
    iterations=500,
    loss_function = 'CrossEntropy'
)
pars = model.grid_search({"learning_rate":10.**np.arange(-3,1)}, X_train, y_train, cv=3, stratified=True, shuffle=True)

0:	learn: 0.6918837	test: 0.6918865	best: 0.6918865 (0)	total: 148ms	remaining: 1m 13s
1:	learn: 0.6906221	test: 0.6906325	best: 0.6906325 (1)	total: 155ms	remaining: 38.5s
2:	learn: 0.6893756	test: 0.6893922	best: 0.6893922 (2)	total: 161ms	remaining: 26.6s
3:	learn: 0.6881257	test: 0.6881486	best: 0.6881486 (3)	total: 168ms	remaining: 20.8s
4:	learn: 0.6868667	test: 0.6868939	best: 0.6868939 (4)	total: 174ms	remaining: 17.2s
5:	learn: 0.6855959	test: 0.6856306	best: 0.6856306 (5)	total: 180ms	remaining: 14.9s
6:	learn: 0.6843676	test: 0.6844033	best: 0.6844033 (6)	total: 187ms	remaining: 13.2s
7:	learn: 0.6831224	test: 0.6831684	best: 0.6831684 (7)	total: 193ms	remaining: 11.9s
8:	learn: 0.6819070	test: 0.6819565	best: 0.6819565 (8)	total: 202ms	remaining: 11s
9:	learn: 0.6806938	test: 0.6807463	best: 0.6807463 (9)	total: 208ms	remaining: 10.2s
10:	learn: 0.6794726	test: 0.6795311	best: 0.6795311 (10)	total: 215ms	remaining: 9.55s
11:	learn: 0.6782490	test: 0.6783116	best: 0.6783116 

105:	learn: 0.5761823	test: 0.5769202	best: 0.5769202 (105)	total: 906ms	remaining: 3.37s
106:	learn: 0.5752315	test: 0.5759763	best: 0.5759763 (106)	total: 919ms	remaining: 3.37s
107:	learn: 0.5742612	test: 0.5750114	best: 0.5750114 (107)	total: 925ms	remaining: 3.36s
108:	learn: 0.5733214	test: 0.5740759	best: 0.5740759 (108)	total: 932ms	remaining: 3.34s
109:	learn: 0.5723789	test: 0.5731391	best: 0.5731391 (109)	total: 941ms	remaining: 3.34s
110:	learn: 0.5714400	test: 0.5722037	best: 0.5722037 (110)	total: 950ms	remaining: 3.33s
111:	learn: 0.5704948	test: 0.5712640	best: 0.5712640 (111)	total: 956ms	remaining: 3.31s
112:	learn: 0.5695663	test: 0.5703389	best: 0.5703389 (112)	total: 964ms	remaining: 3.3s
113:	learn: 0.5686328	test: 0.5694109	best: 0.5694109 (113)	total: 970ms	remaining: 3.28s
114:	learn: 0.5677062	test: 0.5684893	best: 0.5684893 (114)	total: 983ms	remaining: 3.29s
115:	learn: 0.5667663	test: 0.5675560	best: 0.5675560 (115)	total: 990ms	remaining: 3.28s
116:	learn:

206:	learn: 0.4917096	test: 0.4929144	best: 0.4929144 (206)	total: 1.65s	remaining: 2.33s
207:	learn: 0.4909779	test: 0.4921885	best: 0.4921885 (207)	total: 1.65s	remaining: 2.32s
208:	learn: 0.4902409	test: 0.4914592	best: 0.4914592 (208)	total: 1.66s	remaining: 2.31s
209:	learn: 0.4895160	test: 0.4907403	best: 0.4907403 (209)	total: 1.67s	remaining: 2.3s
210:	learn: 0.4888003	test: 0.4900298	best: 0.4900298 (210)	total: 1.68s	remaining: 2.29s
211:	learn: 0.4880783	test: 0.4893103	best: 0.4893103 (211)	total: 1.68s	remaining: 2.29s
212:	learn: 0.4873422	test: 0.4885775	best: 0.4885775 (212)	total: 1.69s	remaining: 2.27s
213:	learn: 0.4866382	test: 0.4878772	best: 0.4878772 (213)	total: 1.7s	remaining: 2.27s
214:	learn: 0.4859067	test: 0.4871512	best: 0.4871512 (214)	total: 1.7s	remaining: 2.26s
215:	learn: 0.4851801	test: 0.4864319	best: 0.4864319 (215)	total: 1.71s	remaining: 2.25s
216:	learn: 0.4844740	test: 0.4857305	best: 0.4857305 (216)	total: 1.72s	remaining: 2.24s
217:	learn: 0

311:	learn: 0.4251496	test: 0.4269359	best: 0.4269359 (311)	total: 2.41s	remaining: 1.45s
312:	learn: 0.4245974	test: 0.4263911	best: 0.4263911 (312)	total: 2.41s	remaining: 1.44s
313:	learn: 0.4240548	test: 0.4258537	best: 0.4258537 (313)	total: 2.42s	remaining: 1.44s
314:	learn: 0.4235219	test: 0.4253239	best: 0.4253239 (314)	total: 2.43s	remaining: 1.43s
315:	learn: 0.4229631	test: 0.4247741	best: 0.4247741 (315)	total: 2.44s	remaining: 1.42s
316:	learn: 0.4224091	test: 0.4242262	best: 0.4242262 (316)	total: 2.44s	remaining: 1.41s
317:	learn: 0.4218721	test: 0.4236947	best: 0.4236947 (317)	total: 2.45s	remaining: 1.4s
318:	learn: 0.4213374	test: 0.4231628	best: 0.4231628 (318)	total: 2.46s	remaining: 1.4s
319:	learn: 0.4208031	test: 0.4226330	best: 0.4226330 (319)	total: 2.46s	remaining: 1.39s
320:	learn: 0.4202720	test: 0.4221048	best: 0.4221048 (320)	total: 2.47s	remaining: 1.38s
321:	learn: 0.4197374	test: 0.4215755	best: 0.4215755 (321)	total: 2.48s	remaining: 1.37s
322:	learn: 

414:	learn: 0.3757943	test: 0.3781500	best: 0.3781500 (414)	total: 3.15s	remaining: 644ms
415:	learn: 0.3753823	test: 0.3777423	best: 0.3777423 (415)	total: 3.15s	remaining: 637ms
416:	learn: 0.3749661	test: 0.3773288	best: 0.3773288 (416)	total: 3.16s	remaining: 629ms
417:	learn: 0.3745566	test: 0.3769224	best: 0.3769224 (417)	total: 3.17s	remaining: 621ms
418:	learn: 0.3741474	test: 0.3765172	best: 0.3765172 (418)	total: 3.17s	remaining: 614ms
419:	learn: 0.3737264	test: 0.3761009	best: 0.3761009 (419)	total: 3.18s	remaining: 606ms
420:	learn: 0.3733126	test: 0.3756919	best: 0.3756919 (420)	total: 3.19s	remaining: 598ms
421:	learn: 0.3728985	test: 0.3752825	best: 0.3752825 (421)	total: 3.19s	remaining: 591ms
422:	learn: 0.3724870	test: 0.3748766	best: 0.3748766 (422)	total: 3.2s	remaining: 583ms
423:	learn: 0.3720855	test: 0.3744767	best: 0.3744767 (423)	total: 3.21s	remaining: 575ms
424:	learn: 0.3716843	test: 0.3740803	best: 0.3740803 (424)	total: 3.22s	remaining: 568ms
425:	learn:

...

58:	learn: 0.1822075	test: 0.1990721	best: 0.1990721 (58)	total: 412ms	remaining: 3.08s
59:	learn: 0.1819229	test: 0.1989362	best: 0.1989362 (59)	total: 418ms	remaining: 3.06s
60:	learn: 0.1815557	test: 0.1988813	best: 0.1988813 (60)	total: 426ms	remaining: 3.07s
61:	learn: 0.1809505	test: 0.1985374	best: 0.1985374 (61)	total: 434ms	remaining: 3.06s
62:	learn: 0.1804605	test: 0.1983640	best: 0.1983640 (62)	total: 441ms	remaining: 3.06s
63:	learn: 0.1801590	test: 0.1982502	best: 0.1982502 (63)	total: 448ms	remaining: 3.05s
64:	learn: 0.1798069	test: 0.1982614	best: 0.1982502 (63)	total: 455ms	remaining: 3.04s
65:	learn: 0.1794300	test: 0.1982489	best: 0.1982489 (65)	total: 466ms	remaining: 3.06s
66:	learn: 0.1789634	test: 0.1981172	best: 0.1981172 (66)	total: 473ms	remaining: 3.05s
67:	learn: 0.1786588	test: 0.1979231	best: 0.1979231 (67)	total: 479ms	remaining: 3.04s
68:	learn: 0.1781833	test: 0.1978876	best: 0.1978876 (68)	total: 487ms	remaining: 3.04s
69:	learn: 0.1779038	test: 0.197

161:	learn: 0.1482204	test: 0.1875083	best: 0.1874297 (159)	total: 1.14s	remaining: 2.39s
162:	learn: 0.1479030	test: 0.1875358	best: 0.1874297 (159)	total: 1.15s	remaining: 2.38s
163:	learn: 0.1478265	test: 0.1875632	best: 0.1874297 (159)	total: 1.16s	remaining: 2.38s
164:	learn: 0.1475846	test: 0.1874403	best: 0.1874297 (159)	total: 1.17s	remaining: 2.37s
165:	learn: 0.1473506	test: 0.1874307	best: 0.1874297 (159)	total: 1.18s	remaining: 2.36s
166:	learn: 0.1472550	test: 0.1874208	best: 0.1874208 (166)	total: 1.18s	remaining: 2.36s
167:	learn: 0.1469688	test: 0.1871879	best: 0.1871879 (167)	total: 1.19s	remaining: 2.36s
168:	learn: 0.1467813	test: 0.1871442	best: 0.1871442 (168)	total: 1.2s	remaining: 2.36s
169:	learn: 0.1464808	test: 0.1870075	best: 0.1870075 (169)	total: 1.21s	remaining: 2.35s
170:	learn: 0.1462907	test: 0.1869139	best: 0.1869139 (170)	total: 1.22s	remaining: 2.35s
171:	learn: 0.1460365	test: 0.1869979	best: 0.1869139 (170)	total: 1.23s	remaining: 2.35s
172:	learn:

260:	learn: 0.1262656	test: 0.1834032	best: 0.1833500 (256)	total: 1.9s	remaining: 1.74s
261:	learn: 0.1261167	test: 0.1834437	best: 0.1833500 (256)	total: 1.9s	remaining: 1.73s
262:	learn: 0.1258844	test: 0.1834742	best: 0.1833500 (256)	total: 1.91s	remaining: 1.72s
263:	learn: 0.1257252	test: 0.1834073	best: 0.1833500 (256)	total: 1.92s	remaining: 1.71s
264:	learn: 0.1255532	test: 0.1833511	best: 0.1833500 (256)	total: 1.92s	remaining: 1.71s
265:	learn: 0.1252574	test: 0.1832176	best: 0.1832176 (265)	total: 1.93s	remaining: 1.7s
266:	learn: 0.1251098	test: 0.1832563	best: 0.1832176 (265)	total: 1.94s	remaining: 1.69s
267:	learn: 0.1250009	test: 0.1832303	best: 0.1832176 (265)	total: 1.94s	remaining: 1.68s
268:	learn: 0.1248793	test: 0.1831659	best: 0.1831659 (268)	total: 1.95s	remaining: 1.68s
269:	learn: 0.1247541	test: 0.1831597	best: 0.1831597 (269)	total: 1.96s	remaining: 1.67s
270:	learn: 0.1247077	test: 0.1831289	best: 0.1831289 (270)	total: 1.97s	remaining: 1.66s
271:	learn: 0

362:	learn: 0.1100833	test: 0.1814645	best: 0.1813418 (345)	total: 2.63s	remaining: 994ms
363:	learn: 0.1099211	test: 0.1814508	best: 0.1813418 (345)	total: 2.64s	remaining: 986ms
364:	learn: 0.1097069	test: 0.1814165	best: 0.1813418 (345)	total: 2.65s	remaining: 980ms
365:	learn: 0.1095392	test: 0.1814605	best: 0.1813418 (345)	total: 2.65s	remaining: 972ms
366:	learn: 0.1094389	test: 0.1814373	best: 0.1813418 (345)	total: 2.66s	remaining: 965ms
367:	learn: 0.1093465	test: 0.1814833	best: 0.1813418 (345)	total: 2.67s	remaining: 957ms
368:	learn: 0.1092142	test: 0.1815039	best: 0.1813418 (345)	total: 2.67s	remaining: 949ms
369:	learn: 0.1090107	test: 0.1812680	best: 0.1812680 (369)	total: 2.68s	remaining: 943ms
370:	learn: 0.1088501	test: 0.1812983	best: 0.1812680 (369)	total: 2.69s	remaining: 935ms
371:	learn: 0.1087660	test: 0.1812617	best: 0.1812617 (371)	total: 2.7s	remaining: 928ms
372:	learn: 0.1087149	test: 0.1812560	best: 0.1812560 (372)	total: 2.7s	remaining: 920ms
373:	learn: 

469:	learn: 0.0971281	test: 0.1801625	best: 0.1801347 (468)	total: 3.55s	remaining: 227ms
470:	learn: 0.0969820	test: 0.1801330	best: 0.1801330 (470)	total: 3.56s	remaining: 219ms
471:	learn: 0.0968351	test: 0.1801423	best: 0.1801330 (470)	total: 3.57s	remaining: 212ms
472:	learn: 0.0967457	test: 0.1801561	best: 0.1801330 (470)	total: 3.58s	remaining: 204ms
473:	learn: 0.0966319	test: 0.1801951	best: 0.1801330 (470)	total: 3.59s	remaining: 197ms
474:	learn: 0.0964708	test: 0.1801682	best: 0.1801330 (470)	total: 3.6s	remaining: 189ms
475:	learn: 0.0963809	test: 0.1801724	best: 0.1801330 (470)	total: 3.61s	remaining: 182ms
476:	learn: 0.0963171	test: 0.1802168	best: 0.1801330 (470)	total: 3.62s	remaining: 175ms
477:	learn: 0.0962335	test: 0.1802051	best: 0.1801330 (470)	total: 3.63s	remaining: 167ms
478:	learn: 0.0961137	test: 0.1802245	best: 0.1801330 (470)	total: 3.64s	remaining: 159ms
479:	learn: 0.0960107	test: 0.1801574	best: 0.1801330 (470)	total: 3.65s	remaining: 152ms
480:	learn:

In [ ]:
pars["params"]

{'learning_rate': 0.1}

In [ ]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    loss_function = 'CrossEntropy'
)

model.fit(X_train_res, y_train_res,
         cat_features=np.arange(9),
        eval_set=(X_val, y_val),
        verbose=50,
         early_stopping_rounds=30)

0:	learn: 0.5774119	test: 0.6103323	best: 0.6103323 (0)	total: 103ms	remaining: 1m 43s
50:	learn: 0.1202203	test: 0.1645173	best: 0.1645173 (50)	total: 5.47s	remaining: 1m 41s
100:	learn: 0.1049028	test: 0.1607736	best: 0.1607544 (99)	total: 12.7s	remaining: 1m 53s
150:	learn: 0.0970496	test: 0.1565546	best: 0.1565005 (149)	total: 19s	remaining: 1m 47s
200:	learn: 0.0912577	test: 0.1548680	best: 0.1548680 (200)	total: 26.4s	remaining: 1m 44s
250:	learn: 0.0867708	test: 0.1538020	best: 0.1537192 (244)	total: 32.8s	remaining: 1m 37s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.1537192389
bestIteration = 244

Shrink model to first 245 iterations.


In [ ]:
print("Score on eval set:")
model.score(X_val, y_val)

Score on eval set:


0.9517292398742371

In [ ]:
pred = model.predict_proba(test.drop(["id"], axis=1))[:,1]
res = pd.concat([test["id"], pd.Series(pred, name="Action")], axis=1)
print(res)
res.to_csv("submissions.csv", index=False)

          id    Action
0          1  0.948168
1          2  0.981004
2          3  0.993221
3          4  0.995473
4          5  0.996577
...      ...       ...
58916  58917  0.991781
58917  58918  0.988894
58918  58919  0.992900
58919  58920  0.961026
58920  58921  0.992259

[58921 rows x 2 columns]


## Kaggle score :
# 0.87788

## Leaders's public score:
# 0.92964